# 2017-04-09 : learning about networking using the zmq python library
----
Reminders : I'm trying to write a pythn-on script that run as background (daemon) on the Pi, that allow to interact with few command lines (API, maybe using zmq and/or socket) to do 3 tasks :
- Ask the camera if it's ready to record
- Allow the camera to answer positively or negatively
- Ask the camera to cut down the connection  


- [ ] Create a daemon : Canceled (Don't need one anymore)

Like last week, I have connection problems to the Pi. But this time, restarting it didn't solved the problem. After talking with L. Perrinet, we didn't found where it come from.  
I'll work on the daemon/API script until further notice.  
More informations about zmq : http://zeromq.org/, http://api.zeromq.org/ and http://zguide.zeromq.org/py:all  
Going further with the interactive python scripts : https://docs.python.org/2/library/thread.html#module-thread and https://docs.python.org/2/library/threading.html  

The connection to the Pi works, it go a new IP : __10.164.7.213__

    ssh pi@10.164.7.213

To learn how zmq is working, I'm reading the (complete) documentation : http://zguide.zeromq.org/py:all  
The zmq python library isn't installed on the pi. Problem : When trying to install it through

    sudo aptitude install python3-zmq

output :

    E: Could not get lock /var/lib/dpkg/lock - open (11: Resource temporarily unavailable)
    E: Unable to lock the administration directory (/var/lib/dpkg/), is another process using it?
    W: Could not lock the cache file; this usually means that dpkg or another apt tool is already installing packages.
    (...)
    
But no other apt process should be running.

    ps aux | grep apt

output 3 apt process running :

    root       747  0.0  0.3   6184  2956 pts/0    S+   14:26   0:00 sudo apt-get upgrade
    root       751  7.3  3.4  33656 30592 pts/0    S+   14:26   3:32 apt-get upgrade
    pi       13293  0.0  0.1   4248  1432 pts/2    S+   15:14   0:00 grep --color=auto apt
    
After verification, they were launched by L. Perrinet. Waiting their end to install zmq.  
zmq python library is succesfully installed on the pi.

The following scripts works on a local machine but doesn't initiate a computer/pi connection :

    '''Server side'''

    import zmq

    context = zmq.Context()
    print('Connecting to client...')
    socket = context.socket(zmq.REQ)
    socket.connect("tcp://localhost:5555")
    print('Client connected')

    for request in range(10):
        print("Sending request %s..." % request)
        socket.send(b"Hello")

        message = socket.recv() #Wait for next request from client
        print("Received reply %s [ %s ]" % (request, message))

    print('No more requests')
  

    '''Client side'''

    import time
    import zmq

    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.bind("tcp://*:5555")
    print('Server connected')

    while True:
        message = socket.recv() #Wait for next request from client
        print("Received request: %s" % message)

        time.sleep(2)
        socket.send(b"World") #Send reply back to client
        
More informations about print(b"sting") : http://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal. It allow the string to be converted into bytes, necessary to be usable for other programming language.  

To make these scripts works with a computer/pi connection, we need to modify the socket.connect parameters, from

    socket.connect("tcp://localhost:5555")
    
to 

    socket.connect("tcp://10.164.7.213:5555")
    
Then pi output :

    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    Received request: Hello
    
While computer output :

    Connecting to client...
    Client connected
    Sending request 0...
    Received reply 0 [ b'World' ]
    Sending request 1...
    Received reply 1 [ b'World' ]
    Sending request 2...
    Received reply 2 [ b'World' ]
    Sending request 3...
    Received reply 3 [ b'World' ]
    Sending request 4...
    Received reply 4 [ b'World' ]
    Sending request 5...
    Received reply 5 [ b'World' ]
    Sending request 6...
    Received reply 6 [ b'World' ]
    Sending request 7...
    Received reply 7 [ b'World' ]
    Sending request 8...
    Received reply 8 [ b'World' ]
    Sending request 9...
    Received reply 9 [ b'World' ]
    No more requests

There can be only one zmq context per process. It'll contain all the sockets of that process. Wrinting two contexts in one script means there's two process from zmq's point of view.  
zmq interact with sockets by applying modifications : open/close, set option/get option, bind/connect and send/receive (recv) messages.  
Client and server use different commands under certain circumstances. __Client__ is usually the most __dynamic node__, and __connect__ its sockets while __server__ is the most __static one__ and __bind__ its sockets, because it have the most stable adress. Here server will be the Pi because it is central to our network (we want to communicate with it from multiple computers).  
zmq allows to start each node independantly, contrary to other network managers that need the server to start first.  
Sockets are open until other notice and once bind/connected, automaticaly accept to send/receive messages (security?). 

Trying to add the picamera interaction into the zmq scripts...  
A problem with the following Pi-sided line seems to induce problems :

    socket.send(struct.pack('<L', stream.tell()))

# 2017-05-10 : merging the zmq and picamera scripts
----

- [X] Solve the file format problem

The computer-sided script output : 

    Received reply 0 [ b'\xf0\xd9\x02\x00' ]
    (...)
    ---> 22     image = Image.open(image_stream)
    (...)
    OSError: cannot identify image file <_io.BytesIO object at 0x7f8417cb8af0>

Reminds me of a previous problem I had with a file format problem : see W2_Progress_and_notes.ipynb/2017-05-05  
But this time, the error is triggered from the first image. The problem may come from a encoding problem at the pi side.  

The Pi-sided script output :

    ---> socket.send(struct.pack('<L', stream.tell()))
    File "socket.pyx", line 574, in zmq.backend.cython.socket.Socket.send (zmq/backend/cython/socket.c:5390)
    File "socket.pyx", line 621, in zmq.backend.cython.socket.Socket.send (zmq/backend/cython/socket.c:5157)
    File "socket.pyx", line 181, in zmq.backend.cython.socket._send_copy (zmq/backend/cython/socket.c:2110)
    File "checkrc.pxd", line 21, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:6103)
    zmq.error.ZMQError: Operation cannot be accomplished in current state

The error may come from an error in the script.  
After some researches, it is a common bug and may be caused by "deadlocks" (see : http://stackoverflow.com/questions/41009900/python-zmq-operation-cannot-be-accomplished-in-current-state)  
As suggested here : http://stackoverflow.com/questions/36391495/deadlock-when-synchronizing-two-simple-python3-scripts-using-0mq-zeromq, I tried to close sockets and contexts, but it did not solved the problem.  

After several hours of research, I'm still unable to solve that bug...  

I tried to get rid of loops inside the Pi-sided script to simplify it, because I was wondering that zmq could only send one answer per request.  
I might be right, because the Pi-sided error message is not showing anymore.  

Now I have a problem with the data that the Pi is sending, as the computer-sided script is outputing :

    (...)
    Sending request 0...
    Received reply 0 [ b'' ]
    (...)
    ---> 25     image = Image.open(image_stream)
    (...)
    OSError: cannot identify image file <_io.BytesIO object at 0x7f10dce2e728>
    
So the Pi reply seems to be an empty bytes string, instead of containing the bytes string component the captured image.

- [X] Solve the empty string reply problem

Scripts modified a bit to send 5 requests from computer and get 5 replies from Pi, and integrated the 'kill yourself' signal that make the Pi cut down the connection. No error except for the empty strings I get as reply. 

Modifying the pi-sided script to show the picture values through 

    print('dbkey n1', stream.getvalue())
    
Allow to see that picture are taken and the stream contain bytes strings. Why are these not sent correclty to the computer?  
The data may be too big to be sent entirely? 

    print('dbkey n1', len(stream.getvalue()))
    
at Pi, output : 

    (...)
    ('dbkey n1', 204193)
    (...)

while at computer, output :

    (...)
    (dbkey n1, 0)
    
More precisly, the byte strings seems to be lost in the Pi-sided script. See :

    (...)
    print(len(stream.getvalue()))
    (...)
    print(len(stream))
    (...)
    print(len(stream.read1(-1)))

# 2017-05-11 : starting to work with OpenRetina scripts
------
Still need to solve the empty string reply problem...  
To resume what I know : the communication through zmq network seem to work correctly, as messages are transmitted from computer to Pi and vice versa. When asked, the picamera take pictures, as shown by :

    stream = io.BytesIO()
    (...)
    camera.capture(stream, format='jpeg')
	print('dbkey n1', len(stream.getvalue()))
    (...)
    
output : 

    ('dbkey n1', 199757)
    
but just after these lines : 

    (...)
	reply = stream.read1(-1)
	print('dbkey n3', len(reply))

	socket.send(reply)
    (...)
    
output :

    ('dbkey n3', 0)

So the problem is created among these lines.  
The error was stupid. After working a day on it, I just replaced 
    
    (...)
    reply = stream.read1(-1)
    (...)

by

    (...)
    reply = stream.getvalue()
    (...)
    
and it finally works.  
The capture colors are a bit weird. Close objects looks fine but some distants objects looks pink instead of purple. It may be interesting to play with camera's saturation and white balance parameters to correct these colors (as suggested here : https://raspberrypi.stackexchange.com/questions/14515/raspi-cam-color-of-image).

__Reverse-engineering__ the OpenRetina project helped me understand how the various scripts are working. My own scripts can be considered as a __low-mimesis model__ of the visual system activity : the __picamera is the retina__ that __catch raw visual data__, the __internet connection__ (ethernet cable then ssh bridge) __is the optic nerve__ __transmitting the data__ and the __computer is the visual nervous central networks__ that __process the data__. 

Now I can start using the OpenRetina scripts directly. The final objective is to make OpenRetina to stop sending whole pictures as bytes arrays as it does now, but instead to send only active bytes locations (so 1 byte per active byte, instead of multiple containing more informations, like color captured).  

L. Perrinet wrote some jupyter notebooks that'll help me to accomodate with the OpenRetina operations. The first one is __using_openretina_testing_noise.ipynb__

- [X] Run the noise testing script

It contain these lines : 

    %load_ext autoreload
    %autoreload 2
    
Allowing to reload modules and refreshing imported libraries at every script start (more on : https://ipython.org/ipython-doc/3/config/extensions/autoreload.html).  

    %%writefile 2017-05-12_using_openretina_testing_noise.py
    
Allow to automaticaly write a file with the line's box content, here a .py file executable by the Pi.

    from openRetina import openRetina
    phrs = openRetina(model=dict(layer='phrs', # label for this layer
                                 input=['noise'], # input: can be the camera, noise, a movie (TODO)
                                 output=['stream'] # output: can be stream, display, ...
                                 ))
    phrs.run()
    
That bloc is the main way to call the OpenRetina content, and allow to easily modify its parameters. Here

    input=['noise']
    
call the noise section of the run() function, generating random data and allowing to check if the program is functionnal and free of errors.

    !ipython3 2017-05-12_using_openretina_testing_noise.py &
    
Previous line allows to run the selected script in background.  
To run these scripts, I need to install OpenRetina on the computer. The classic command line :

    git clone https://github.com/laurentperrinet/openRetina
    
worked and I got all the folder.  
But it seems that I can't run :

    from openRetina import openRetina
    
because I got the output : 

    (...)
    ----> 3 from openRetina import openRetina

    ImportError: cannot import name 'openRetina'
    
I also add to (re?)install numpy for python3:

    pip3 install numpy

Replacing the following lines seems to solve the importing problem:

    from openRetina import openRetina
    
by: 

    import openRetina 
 
But, when running : 

    !ipython3 2017-05-12_using_openretina_testing_noise_srv.py
    
I got this error as an output :

    ---> 14                                  T_SIM=20))
         15 noise.run()

    TypeError: 'module' object is not callable
    Traceback (most recent call last):
      File "./2017-05-12_using_openretina_testing_noise.py", line 14, in <module>
        output=['stream'] # output: can be stream, display, ...
    TypeError: 'module' object is not callable

Maybe the import problem is not completly resolved? Or using_openretina_testing_noise.py contain a mistake?


# 2017-05-12
----

I moved all my scripts from ./Documents to ./Documents/Intership to clean my directories.

- [X] Solve the uncallable module problem

The problem is present as soon as we call using_openretina_testing_noise.py, as it output the same error as described previously.  
As suggested here : http://stackoverflow.com/questions/4534438/typeerror-module-object-is-not-callable, this problem can be caused by remplacing "from openRetina import openRetina" by 'import openRetina". But again, cancelling this makes the ImportError to come back.  

Some internet pages suggest that I may be induced by a circular import dependencies, not permitted in python (see : http://stackoverflow.com/questions/17845366/importerror-cannot-import-name and http://stackoverflow.com/questions/1556387/circular-import-dependency-in-python).  
After some researches, it seems that it may be induced by the openRetina folder being outside the sys.path where python modules are located. So I tried to move the openRetina folder to that path, using : 

    cd ../
    sudo mv -v openRetina /usr/lib/python3.5/

then inside the openRetina folder, move the src folder content to main. After doing this, errors are still here but there is some evolution. Calling :

    !ipython3 2017-05-12_using_openretina_testing_noise.py
    
output : 

    ]0;IPython: Documents/Intership💀  Could not load visualisation
    Running retina on port:  5566
    
and calling : 

    !ipython3 2017-05-12_using_openretina_testing_noise_srv.py
    
ouput : 

    ]0;IPython: Documents/Intership💀  Could not load visualisation
    ---------------------------------------------------------------------------
    TypeError                                 Traceback (most recent call last)
    /home/albert/Documents/Intership/2017-05-12_using_openretina_testing_noise_srv.py in <module>()
         12                                  input=['stream'], # input: can be the camera, noise, a movie (TODO)
         13                                  output=['display'], # output: can be stream, display, capture,...
    ---> 14                                  T_SIM=20))
         15 noise.run()

    TypeError: 'module' object is not callable
    💀  Could not load visualisation
    Running retina on port:  5566

The errors are still there when doing a clean install :

    cd ./Documents/openRetina/
    sudo pip3 -e .
    
The problem don't come from the network, because the host ip isn't defined when calling so the following line automaticaly put it as localhost :

     if not 'ip' in self.model.keys(): self.model['ip']='localhost'
     
 I tried to run another script (2017-05-12_using_openretina_testing_local.ipynb) and it shows another error, I'll have to work on it later.
 
I had to install some additionnal libraries the script needed :

    pip3 install vispy --user
    pip3 install pyglet --user
    
After installing these modules and reversing "import openRetina" to "from openRetina import openRetina", the script seem to work. A window poped up and showed 20s of noise signal + got multiple outputs for server (srv) side:

    (...)
    Sending request
    Received request b'REQ'
    Image   (1280, 720, 3) 0 254
    (...)

But something doesn't launch correctly because I have to interrupt the kernel to make the window appear. Otherwise the script seem to wait undefinitely. In addition, I got this output for srv, but it doesn't seem to bother the script execution (more information :http://stackoverflow.com/questions/33571421/ipython-notebook-shimwarning-the-ipython-kernel-package-has-been-deprecated) :

    /home/albert/.local/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
      "You should import from traitlets.config instead.", ShimWarning)
     
It was provoqued by a human error : I was launching the client (*_using_openretina_testing_noise.py*) and the server (*_using_openretina_testing_noise-srv.py*) scripts, but the last contain the lines 

    import subprocess
    p = subprocess.Popen(['./2017-05-12_using_openretina_testing_noise.py'])
    
that also run the client script, so the first one was waiting for a request that didn't come (more informations about the subprocess module : https://docs.python.org/2/library/subprocess.html)

- [ ] Run the local testing script

Next step it to run *_using_openretina_testing_local*.  
When running, a window is poping-up but only show a black picture, so the send/rcv system seem to work, as these outputs suggest : 

    (...)
    Sending request
    Received request b'REQ'
    (...)

but the capture system may not work correctly, as these error messages suggest :

    (...)
    File "./2017-05-12_using_openretina_testing_camera.py", line 15, in <module>
        phrs.run()
      File "/home/albert/Documents/openRetina/src/openRetina.py", line 175, in run
        self.camera.grab()
      File "/home/albert/Documents/openRetina/src/openRetina.py", line 93, in grab
        ret, frame_bgr = self.cap.read()
    AttributeError: 'PhotoReceptor' object has no attribute 'cap'
    (...)
    
and it force me to manually interrupt the kernel to close the window.  
The python3 version for opencv wasn't installed on the computer (I only used the python2 version until now) :

    pip3 install opencv-python --user
    
It didn't completly solved the problem, but the error message changed :

      File "./2017-05-12_using_openretina_testing_camera.py", line 15, in <module>
        phrs.run()
      File "/home/albert/Documents/openRetina/src/openRetina.py", line 175, in run
        self.camera.grab()
      File "/home/albert/Documents/openRetina/src/openRetina.py", line 94, in grab
        self.output = frame_bgr[:, :, ::-1]     #BGR to RGB
    TypeError: 'NoneType' object is not subscriptable
    
Modified the concerned lines to match the program (*camera.py*) I wrote, from :

            ret, frame_bgr = self.cap.read()
            self.output = frame_bgr[:, :, ::-1]     #BGR to RGB
    #return frame
    
to : 

            ret, frame_bgr = self.cap.read()
            #self.output = frame_bgr[:, :, ::-1]     #BGR to RGB
            frame = frame_bgr
    return frame
    
Only made the error jump to other lines, as the script now output : 

      File "./2017-05-12_using_openretina_testing_camera.py", line 15, in <module>
        phrs.run()
      File "/home/albert/Documents/openRetina/src/openRetina.py", line 180, in run
        data = self.code(self.camera.output)
    AttributeError: 'PhotoReceptor' object has no attribute 'output'
    
Redefining the output attribute by going from :

    (...)
    self.camera.grab()
    (...)
    data = self.code(self.camera.output)
    (...)
                
to : 

    (...)
    self.frame = self.camera.grab()
    (...)
    data = self.code(self.frame)
    (...)
    
Again, the error jumped to further lines, as the script now output :

        File "./2017-05-12_using_openretina_testing_camera.py", line 15, in <module>
        phrs.run()
      File "/home/albert/Documents/openRetina/src/openRetina.py", line 180, in run
        data = self.code(self.frame)
      File "/home/albert/Documents/openRetina/src/openRetina.py", line 245, in code
        image = image.astype(np.float)
    AttributeError: 'NoneType' object has no attribute 'astype'

# outline
------

- use the openRetina library to visualize noise
- use the openRetina library to replicate what was done in W1 & 2

